<a href="https://colab.research.google.com/github/ShreyBh/Fake-News-Classifier-Using-Bidirectional-LSTM/blob/main/FakeNewsClassifierUsingBidirectionalLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Fake News Classifier Using Bidirectional LSTM***

Dataset : https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [4]:
df=pd.read_csv('train.csv')

In [5]:
df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df=df.dropna() ##DROP Nan Values

In [7]:
X=df.drop('label',axis=1) ## Get the Independent Features

In [8]:
y=df['label'] ## Get the Dependent features

In [9]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape


(18285,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.8.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [15]:
voc_size=5000 ### Vocabulary size

**Onehot Representation**

In [16]:
messages=X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[341, 4891, 4932, 2419, 4919, 3039, 1542, 1615, 1449, 4349],
 [4295, 4035, 3454, 3433, 1202, 4285, 818],
 [2279, 3392, 2437, 2474],
 [3229, 1361, 1815, 4197, 1527, 885],
 [74, 1202, 4565, 2540, 4131, 412, 1202, 17, 3045, 2631],
 [3105,
  366,
  2851,
  267,
  4615,
  2958,
  3923,
  1992,
  406,
  134,
  3308,
  822,
  2918,
  2819,
  818],
 [1265, 1321, 4975, 1174, 3492, 3394, 2445, 1024, 4215, 4233, 3900],
 [3307, 1779, 4315, 4713, 2348, 1109, 2958, 4610, 4215, 4233, 3900],
 [1124, 4188, 646, 2739, 1449, 96, 558, 1130, 2958, 4411],
 [4786, 1528, 1529, 3512, 1192, 4702, 2042, 421],
 [185, 1889, 875, 2399, 3155, 3429, 190, 1478, 4600, 2001, 3652],
 [4197, 1595, 4919, 96, 2958, 2348],
 [1771, 2909, 3364, 4741, 1126, 4963, 399, 2444, 847],
 [2723, 1350, 3941, 1538, 2320, 2905, 4306, 4215, 4233, 3900],
 [4252, 2574, 4341, 1291, 755, 4215, 4233, 3900],
 [3199, 3504, 1316, 2150, 4856, 4174, 3801, 2435, 2600, 2751],
 [1675, 2925, 4035],
 [33, 1661, 604, 375, 2958, 2074, 205, 818],
 [4588, 4

**Embedding Representation**



In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1615 1449 4349]
 [   0    0    0 ... 1202 4285  818]
 [   0    0    0 ... 3392 2437 2474]
 ...
 [   0    0    0 ... 4215 4233 3900]
 [   0    0    0 ... 4086 3471 2349]
 [   0    0    0 ... 2890 1548 4681]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  341,
       4891, 4932, 2419, 4919, 3039, 1542, 1615, 1449, 4349], dtype=int32)

In [26]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
len(embedded_docs),y.shape

(18285, (18285,))

In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

**Model Training**

In [32]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 17s 58ms/step - loss: 0.2926 - accuracy: 0.8601 - val_loss: 0.2018 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 8s 43ms/step - loss: 0.1400 - accuracy: 0.9471 - val_loss: 0.1995 - val_accuracy: 0.9178
Epoch 3/10
192/192 [==============================] - 9s 49ms/step - loss: 0.1011 - accuracy: 0.9641 - val_loss: 0.2317 - val_accuracy: 0.9100
Epoch 4/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0773 - accuracy: 0.9740 - val_loss: 0.2487 - val_accuracy: 0.9120
Epoch 5/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0640 - accuracy: 0.9787 - val_loss: 0.2934 - val_accuracy: 0.9123
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0447 - accuracy: 0.9860 - val_loss: 0.3406 - val_accuracy: 0.9117
Epoch 7/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0322 - accuracy: 0.9894 - val_loss: 0.3936 - val_accuracy: 0.90

**Performance Metrics And Accuracy**

In [50]:
y_pred1=(model1.predict(X_test)>0.5).astype("int32")

In [51]:
from sklearn.metrics import confusion_matrix

In [54]:
confusion_matrix(y_test,y_pred1)

array([[3118,  301],
       [ 234, 2382]])

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9113504556752279